<a href="https://colab.research.google.com/github/synergit/adversarial-robustness-toolbox/blob/development_issue_1331/sign_opt_CIFAR10_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: http://home.mit.bme.hu/~hadhazi/Oktatas/NN18/dem3/html_demo/CIFAR-10Demo.html

In [49]:
pip install adversarial-robustness-toolbox

In [50]:
# art.utils only loads first 5000 training and 500 test data
from art.utils import load_cifar10
raw_image = False
(x_train, y_train), (x_test, y_test), min_, max_ = load_cifar10(raw=raw_image)#load_dataset(str("cifar10"))
# x_train.shape (5000, 32, 32, 3)
# y_train.shape (5000, 10)
x_train, y_train = x_train[:5000], y_train[:5000]
# x_test.shape (500, 32, 32, 3)
# y_test.shape (500, 10)
x_test, y_test = x_test[:500], y_test[:500]
# im_shape (32, 32, 3)
im_shape = x_train[0].shape

In [51]:
import matplotlib.pyplot as plt
if raw_image:
  cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
  print('Example training images and their labels: ' + str([x[0] for x in y_train[0:5]])) 
  print('Corresponding classes for the labels: ' + str([cifar_classes[x[0]] for x in y_train[0:5]]))

  f, axarr = plt.subplots(1, 5)
  f.set_size_inches(16, 6)

  for i in range(5):
      img = x_train[i]
      axarr[i].imshow(img)
  plt.show()
else:
  print('change `raw_image` to be True for showing images')

change `raw_image` to be True for showing images


In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define the neural network model, return logits instead of activation in forward method
###
# define the network as https://arxiv.org/pdf/1608.04644.pdf, table 1
###
class Net_table1_CIFAR10(nn.Module):
    def __init__(self):
        super(Net_table1_CIFAR10, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1)
        self.conv_4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1)
        # https://discuss.pytorch.org/t/calculation-for-the-input-to-the-fully-connected-layer/82774/11
        # x.shape: torch.Size([2, 128, 5, 5]) so, 128*5*5
        self.fc_1 = nn.Linear(in_features= 3200, out_features=256) 
        self.fc_2 = nn.Linear(in_features=256, out_features=10)
        
    # https://github.com/Carco-git/CW_Attack_on_MNIST/blob/master/MNIST_Model.py
    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_3(x))
        x = F.relu(self.conv_4(x))
        x = F.max_pool2d(x, 2, 2)
        # print(x.shape) # torch.Size([2, 128, 5, 5])
        x = x.view(-1, 3200)
        
        x = F.relu(self.fc_1(x))
        x = F.dropout(x, p=0.5)
        x = self.fc_2(x)
        return x

# Create the model

model_table1 = Net_table1_CIFAR10()

from torchsummary import summary
model_scratch = Net_table1_CIFAR10()
print(summary(model_scratch,(3, 32, 32)))
"""
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
================================================================
            Conv2d-1           [-1, 64, 30, 30]           1,792
            Conv2d-2           [-1, 64, 28, 28]          36,928
            Conv2d-3          [-1, 128, 12, 12]          73,856
            Conv2d-4          [-1, 128, 10, 10]         147,584
            Linear-5                  [-1, 256]       1,638,656
            Linear-6                   [-1, 10]           2,570
================================================================
Total params: 1,901,386
Trainable params: 1,901,386
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.06
Params size (MB): 7.25
Estimated Total Size (MB): 8.33
----------------------------------------------------------------
None
"""

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 30, 30]           1,792
            Conv2d-2           [-1, 64, 28, 28]          36,928
            Conv2d-3          [-1, 128, 12, 12]          73,856
            Conv2d-4          [-1, 128, 10, 10]         147,584
            Linear-5                  [-1, 256]         819,456
            Linear-6                   [-1, 10]           2,570
Total params: 1,082,186
Trainable params: 1,082,186
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.06
Params size (MB): 4.13
Estimated Total Size (MB): 5.20
----------------------------------------------------------------
None


'\n----------------------------------------------------------------\n        Layer (type)               Output Shape         Param #\n================================================================\n            Conv2d-1           [-1, 64, 30, 30]           1,792\n            Conv2d-2           [-1, 64, 28, 28]          36,928\n            Conv2d-3          [-1, 128, 12, 12]          73,856\n            Conv2d-4          [-1, 128, 10, 10]         147,584\n            Linear-5                  [-1, 256]       1,638,656\n            Linear-6                   [-1, 10]           2,570\n================================================================\nTotal params: 1,901,386\nTrainable params: 1,901,386\nNon-trainable params: 0\n----------------------------------------------------------------\nInput size (MB): 0.01\nForward/backward pass size (MB): 1.06\nParams size (MB): 7.25\nEstimated Total Size (MB): 8.33\n----------------------------------------------------------------\nNone\n'

In [ ]:
import numpy as np
from art.estimators.classification import PyTorchClassifier
criterion = nn.CrossEntropyLoss()
optimizer_table1 = optim.SGD(model_table1.parameters(), lr=0.01, momentum=0.9)

# Create the ART classifier

classifier_table1 = PyTorchClassifier(
    model=model_table1,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer_table1,
    input_shape=(3, 32, 32),
    nb_classes=10,
    device_type = "gpu",
)

# Swap axes to PyTorch's NCHW format

# print(x_train.shape)
x_train = np.transpose(x_train, (0, 3, 2, 1)).astype(np.float32)
# print(f'after {x_train.shape}')
x_test = np.transpose(x_test, (0, 3, 2, 1)).astype(np.float32)

ML_model_Filename = "Pytorch_Model_CIFAR10_table1.pth"

# Load the Model back from file
# try:
#     with open(ML_model_Filename, 'rb') as file:  
#         # classifier_table1 = pickle.load(file)
#         classifier_table1 = torch.load(file)
# except FileNotFoundError:
    # print('No existing model, training the model instead')
print(x_train.shape)
print(y_train.shape)
classifier_table1.fit(x_train, y_train, batch_size=128, nb_epochs=50)
    # Save the model
    # with open(ML_model_Filename, 'wb') as file:  
    #     # pickle.dump(classifier_table1, file)
    #     torch.save(classifier_table1, file)

(5000, 3, 32, 32)
(5000, 10)
